In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import subprocess
import pdb
import astropy.units as u
import astropy.constants as const

In [21]:
def calc_tau(semi, planet_mass, star_mass, Q_planet):
    period = np.sqrt( (semi**3 * 4 * np.pi**2) / (const.G * (planet_mass + star_mass)) )
    n = 1/period
    tau = 1/(Q_planet * n)
    return tau.to(u.s).value
    

In [24]:
# lets make the ctl files
eqtide_cpl_dir = './final_data/runs_eq_only/'
eqtide_ctl_dir = './final_data/eqtide_ctl/'

i = 0
n = 25000
while i <= n-1:
    folder_name = "%05i"%(i)
    working_dir = eqtide_cpl_dir + folder_name
    #copy input files
    mkdir_command = 'mkdir '+eqtide_ctl_dir+folder_name+'/'
    copy_command = 'cp ' + working_dir + '/*.in '+eqtide_ctl_dir+folder_name+'/'
    mkdir = copy = subprocess.call(mkdir_command, shell=True, cwd = './')
    copy = subprocess.call(copy_command, shell=True, cwd = './')
    
    b_file = open(eqtide_cpl_dir + folder_name+'/b.in','r')
    b_content = b_file.readlines()
    star_file = open(eqtide_cpl_dir + folder_name+'/star.in','r')
    star_content = star_file.readlines()
    Q = float(b_content[-5].split('\t\t')[-1][:-1])
    semi = float(b_content[10].split('\t\t')[-1][:-1]) * u.AU
    planet_mass = float(b_content[4].split('\t\t')[-1][:-1])*-1 * u.earthMass
    star_mass = float(star_content[3].split('\t\t')[-1][:-1]) * u.solMass
    tau = calc_tau(semi, planet_mass, star_mass, Q)
    tau_string = 'dTidalTau\t' + str(tau*-1) + '\n'
    b_content[-5] = tau_string
    b_content[2] = 'sTideModel\tt8\n'
    star_content[-6] = 'dTidalTau\t-100\n'
    
    new_b_file = open(eqtide_ctl_dir + folder_name + '/b.in','w')
    new_b_file.write(''.join(b_content))
    new_star_file = open(eqtide_ctl_dir + folder_name + '/star.in','w')
    new_star_file.write(''.join(star_content))
    
    star_file.close()
    b_file.close()
    new_b_file.close()
    new_star_file.close()
    
    
    #pdb.set_trace()
    
    i +=1



In [28]:
s = 0
while s <= n-1:
    name_idx = '%05i'%s
    folder_name = name_idx
    working_directory = eqtide_ctl_dir+folder_name+'/'
    p = subprocess.call(['vplanet vpl.in > log'], shell=True, cwd = working_directory)
    perc = str((s/n)*100) + '%'
    print(s, perc, end='\r')
    s += 1
    continue

In [32]:
n_merges = 0
a_merges = np.array([])
i = 0
while i <= n-1:
    name_idx = '%05i'%i
    wd = eqtide_ctl_dir + name_idx
    log_file = open(wd+'/log','r')
    log_content = log_file.read()
    bMerge = log_content.find('Merge')
    if bMerge != -1:
        n_merges += 1
        a_merges = np.append(a_merges, name_idx)
    i +=1
    
print (n_merges)
a_merges

320


array(['00053', '00124', '00196', '00229', '00332', '00375', '00430',
       '00456', '00470', '00652', '00682', '00849', '01055', '01193',
       '01294', '01315', '01365', '01413', '01479', '01547', '01570',
       '01600', '01620', '01677', '01752', '01836', '01889', '01956',
       '01990', '02137', '02216', '02225', '02242', '02352', '02454',
       '02516', '02548', '02673', '02741', '02787', '02834', '02946',
       '03009', '03071', '03140', '03275', '03312', '03434', '03460',
       '03595', '03596', '03640', '03696', '03736', '03749', '03863',
       '03936', '04075', '04175', '04201', '04204', '04235', '04273',
       '04375', '04382', '04427', '04454', '04564', '04649', '04707',
       '04966', '05121', '05130', '05209', '05275', '05289', '05335',
       '05355', '05451', '05493', '05529', '05662', '05664', '05805',
       '05872', '06027', '06196', '06205', '06237', '06326', '06382',
       '06384', '06610', '06774', '06808', '06826', '06913', '07000',
       '07116', '071